In [8]:
# Загружаем отфильтрованные данные из файла, созданного в 01_initial_filtering.ipynb
import os
import pandas as pd

# Путь к отфильтрованному файлу (относительно текущей директории ноутбука)
filtered_data_path = os.path.join('..', 'agab_filtered.parquet')

try:
    agab_df = pd.read_parquet(filtered_data_path, engine='pyarrow')
    print(f"Загружены отфильтрованные данные из {filtered_data_path}")
    print(f"Размер данных: {agab_df.shape}")
    print(f"Колонки: {list(agab_df.columns)}")
except FileNotFoundError:
    raise FileNotFoundError(
        f"Файл {filtered_data_path} не найден.\n"
        "Пожалуйста, сначала запустите ноутбук 01_initial_filtering.ipynb для создания отфильтрованных данных."
    )

Загружены отфильтрованные данные из ../agab_filtered.parquet
Размер данных: (1112523, 11)
Колонки: ['dataset', 'heavy_sequence', 'light_sequence', 'scfv', 'affinity_type', 'affinity', 'antigen_sequence', 'confidence', 'nanobody', 'metadata', 'processed_measurement']


In [ ]:
# Выведем 1000 случайных строк из agab_df
agab_df.sample(n=1000, random_state=42).to_csv('sample_data.csv', index=False)


In [10]:
# 1. Анализ структуры данных - общая информация о affinity_type
print("=" * 80)
print("АНАЛИЗ ПРЕОБРАЗОВАНИЯ affinity → processed_measurement")
print("=" * 80)

print("\n1. Уникальные значения affinity_type:")
unique_types = agab_df['affinity_type'].unique()
print(f"Всего уникальных типов: {len(unique_types)}")
type_counts = agab_df['affinity_type'].value_counts()
for i, (aff_type, count) in enumerate(type_counts.items(), 1):
    percentage = (count / len(agab_df)) * 100
    print(f"  {i:2d}. {aff_type:25s}: {count:8,} записей ({percentage:5.2f}%)")

print("\n2. Типы данных:")
print(f"  affinity: {agab_df['affinity'].dtype}")
print(f"  processed_measurement: {agab_df['processed_measurement'].dtype}")


АНАЛИЗ ПРЕОБРАЗОВАНИЯ affinity → processed_measurement

1. Уникальные значения affinity_type:
Всего уникальных типов: 10
   1. fuzzy                    :  524,346 записей (47.13%)
   2. bool                     :  289,382 записей (26.01%)
   3. -log KD                  :  152,401 записей (13.70%)
   4. alphaseq                 :  131,645 записей (11.83%)
   5. kd                       :    6,884 записей ( 0.62%)
   6. log_enrichment           :    3,452 записей ( 0.31%)
   7. delta_g                  :    2,725 записей ( 0.24%)
   8. ddg                      :      670 записей ( 0.06%)
   9. elisa_mut_to_wt_ratio    :      658 записей ( 0.06%)
  10. ic_50                    :      360 записей ( 0.03%)

2. Типы данных:
  affinity: object
  processed_measurement: object


In [11]:
# 2. Анализ для каждого affinity_type - примеры пар (affinity, processed_measurement)
print("=" * 80)
print("ПРИМЕРЫ ПРЕОБРАЗОВАНИЙ ПО КАЖДОМУ ТИПУ")
print("=" * 80)

for aff_type in sorted(unique_types):
    subset = agab_df[agab_df['affinity_type'] == aff_type].copy()
    print(f"\n{'='*80}")
    print(f"Тип: {aff_type} (всего записей: {len(subset):,})")
    print(f"{'='*80}")
    
    # Берем 10 случайных примеров
    sample = subset[['affinity', 'processed_measurement']].sample(
        n=min(10, len(subset)), 
        random_state=42
    )
    
    print("\nПримеры пар (affinity → processed_measurement):")
    for idx, row in sample.iterrows():
        print(f"  {row['affinity']:30s} → {row['processed_measurement']}")
    
    # Статистика по типам данных
    print(f"\nТипы значений affinity:")
    affinity_numeric = pd.to_numeric(subset['affinity'], errors='coerce')
    affinity_numeric_count = affinity_numeric.notna().sum()
    affinity_non_numeric_count = affinity_numeric.isna().sum()
    print(f"  Числовые: {affinity_numeric_count:,} ({affinity_numeric_count/len(subset)*100:.1f}%)")
    print(f"  Нечисловые: {affinity_non_numeric_count:,} ({affinity_non_numeric_count/len(subset)*100:.1f}%)")
    
    print(f"\nТипы значений processed_measurement:")
    pm_numeric = pd.to_numeric(subset['processed_measurement'], errors='coerce')
    pm_numeric_count = pm_numeric.notna().sum()
    pm_non_numeric_count = pm_numeric.isna().sum()
    print(f"  Числовые: {pm_numeric_count:,} ({pm_numeric_count/len(subset)*100:.1f}%)")
    print(f"  Нечисловые: {pm_non_numeric_count:,} ({pm_non_numeric_count/len(subset)*100:.1f}%)")
    
    # Если оба числовые - показать статистику
    if affinity_numeric_count > 0 and pm_numeric_count > 0:
        numeric_subset = subset[affinity_numeric.notna() & pm_numeric.notna()].copy()
        numeric_subset['affinity_num'] = pd.to_numeric(numeric_subset['affinity'])
        numeric_subset['pm_num'] = pd.to_numeric(numeric_subset['processed_measurement'])
        
        print(f"\nСтатистика для числовых значений ({len(numeric_subset):,} записей):")
        print(f"  affinity: min={numeric_subset['affinity_num'].min():.4f}, "
              f"max={numeric_subset['affinity_num'].max():.4f}, "
              f"mean={numeric_subset['affinity_num'].mean():.4f}")
        print(f"  processed_measurement: min={numeric_subset['pm_num'].min():.4f}, "
              f"max={numeric_subset['pm_num'].max():.4f}, "
              f"mean={numeric_subset['pm_num'].mean():.4f}")


ПРИМЕРЫ ПРЕОБРАЗОВАНИЙ ПО КАЖДОМУ ТИПУ

Тип: -log KD (всего записей: 152,401)

Примеры пар (affinity → processed_measurement):
  8.443405244432915              → 7.2217026222164575
  9.521655164840006              → 7.760827582420003
  10.06034008167947              → 9.784254257819356
  9.205622897341348              → 7.602811448670674
  9.313579587390398              → 7.656789793695199
  6.0                            → 7.745919185731322
  9.294904634688884              → 7.647452317344442
  6.0                            → 7.796480078866114
  6.0                            → 7.663284902894056
  8.573108757822352              → 7.286554378911176

Типы значений affinity:
  Числовые: 152,401 (100.0%)
  Нечисловые: 0 (0.0%)

Типы значений processed_measurement:
  Числовые: 152,401 (100.0%)
  Нечисловые: 0 (0.0%)

Статистика для числовых значений (152,401 записей):
  affinity: min=5.5979, max=12.4296, mean=7.7631
  processed_measurement: min=5.5979, max=12.4296, mean=7.7631

Тип: alpha

In [12]:
# 3. Детальный анализ для поиска правил преобразования
# Для каждого типа проверяем, есть ли четкая связь между affinity и processed_measurement

import numpy as np

print("=" * 80)
print("ДЕТАЛЬНЫЙ АНАЛИЗ ПРАВИЛ ПРЕОБРАЗОВАНИЯ")
print("=" * 80)

rules = {}

for aff_type in sorted(unique_types):
    subset = agab_df[agab_df['affinity_type'] == aff_type].copy()
    
    print(f"\n{'='*80}")
    print(f"Анализ типа: {aff_type}")
    print(f"{'='*80}")
    
    # Преобразуем в числовой формат где возможно
    subset['affinity_num'] = pd.to_numeric(subset['affinity'], errors='coerce')
    subset['pm_num'] = pd.to_numeric(subset['processed_measurement'], errors='coerce')
    
    numeric_mask = subset['affinity_num'].notna() & subset['pm_num'].notna()
    numeric_subset = subset[numeric_mask].copy()
    
    non_numeric_mask = subset['affinity_num'].isna() | subset['pm_num'].isna()
    non_numeric_subset = subset[non_numeric_mask].copy()
    
    print(f"\nЧисловые пары: {len(numeric_subset):,} ({len(numeric_subset)/len(subset)*100:.1f}%)")
    print(f"Нечисловые пары: {len(non_numeric_subset):,} ({len(non_numeric_subset)/len(subset)*100:.1f}%)")
    
    if len(numeric_subset) > 0:
        # Проверяем разные гипотезы преобразования
        aff = numeric_subset['affinity_num'].values
        pm = numeric_subset['pm_num'].values
        
        # Гипотеза 1: прямое равенство
        equal = np.isclose(aff, pm, rtol=1e-10)
        equal_count = equal.sum()
        
        # Гипотеза 2: отрицательный логарифм (-log)
        # Если affinity > 0, то -log(affinity) может дать processed_measurement
        neg_log_match = 0
        if (aff > 0).all():
            neg_log_pm = -np.log10(aff)
            neg_log_match = np.isclose(neg_log_pm, pm, rtol=1e-5).sum()
        
        # Гипотеза 3: логарифм (log)
        log_match = 0
        if (aff > 0).all():
            log_pm = np.log10(aff)
            log_match = np.isclose(log_pm, pm, rtol=1e-5).sum()
        
        print(f"\nГипотезы преобразования для числовых значений:")
        print(f"  1. Прямое равенство (affinity == processed_measurement): {equal_count:,} совпадений ({equal_count/len(numeric_subset)*100:.1f}%)")
        if (aff > 0).all():
            print(f"  2. Отрицательный логарифм (-log10(affinity)): {neg_log_match:,} совпадений ({neg_log_match/len(numeric_subset)*100:.1f}%)")
            print(f"  3. Логарифм (log10(affinity)): {log_match:,} совпадений ({log_match/len(numeric_subset)*100:.1f}%)")
        
        # Показываем примеры, которые НЕ совпадают напрямую
        if equal_count < len(numeric_subset):
            print(f"\nПримеры НЕпрямого преобразования (первые 5):")
            diff_mask = ~equal
            diff_examples = numeric_subset[diff_mask][['affinity', 'processed_measurement', 'affinity_num', 'pm_num']].head(5)
            for idx, row in diff_examples.iterrows():
                ratio = row['pm_num'] / row['affinity_num'] if row['affinity_num'] != 0 else np.nan
                diff = row['pm_num'] - row['affinity_num']
                print(f"  {row['affinity']:20s} → {row['processed_measurement']:20s} "
                      f"(разница: {diff:.4f}, отношение: {ratio:.4f})")
    
    # Анализ нечисловых значений
    if len(non_numeric_subset) > 0:
        print(f"\nАнализ нечисловых значений (первые 10 примеров):")
        examples = non_numeric_subset[['affinity', 'processed_measurement']].head(10)
        for idx, row in examples.iterrows():
            print(f"  {str(row['affinity']):30s} → {str(row['processed_measurement']):30s}")
        
        # Уникальные комбинации для нечисловых
        unique_combos = non_numeric_subset[['affinity', 'processed_measurement']].drop_duplicates()
        print(f"\nУникальных нечисловых комбинаций: {len(unique_combos)}")
        if len(unique_combos) <= 20:
            print("Все комбинации:")
            for idx, row in unique_combos.iterrows():
                print(f"  {str(row['affinity']):30s} → {str(row['processed_measurement']):30s}")


ДЕТАЛЬНЫЙ АНАЛИЗ ПРАВИЛ ПРЕОБРАЗОВАНИЯ

Анализ типа: -log KD

Числовые пары: 152,401 (100.0%)
Нечисловые пары: 0 (0.0%)

Гипотезы преобразования для числовых значений:
  1. Прямое равенство (affinity == processed_measurement): 18,851 совпадений (12.4%)
  2. Отрицательный логарифм (-log10(affinity)): 0 совпадений (0.0%)
  3. Логарифм (log10(affinity)): 0 совпадений (0.0%)

Примеры НЕпрямого преобразования (первые 5):
  9.475657999505575    → 7.737828999752788    (разница: -1.7378, отношение: 0.8166)
  6.0                  → 7.737828999752788    (разница: 1.7378, отношение: 1.2896)
  9.443128354620542    → 7.721564177310271    (разница: -1.7216, отношение: 0.8177)
  6.0                  → 7.721564177310271    (разница: 1.7216, отношение: 1.2869)
  9.495338597507892    → 7.747669298753946    (разница: -1.7477, отношение: 0.8159)

Анализ типа: alphaseq

Числовые пары: 129,079 (98.1%)
Нечисловые пары: 2,566 (1.9%)

Гипотезы преобразования для числовых значений:
  1. Прямое равенство (affini

In [13]:
# 4. Специальный анализ для типов с нечисловыми значениями (fuzzy, bool)
print("=" * 80)
print("АНАЛИЗ СПЕЦИАЛЬНЫХ ТИПОВ: fuzzy и bool")
print("=" * 80)

for special_type in ['fuzzy', 'bool']:
    if special_type not in unique_types:
        continue
        
    subset = agab_df[agab_df['affinity_type'] == special_type].copy()
    print(f"\n{'='*80}")
    print(f"Тип: {special_type} (всего: {len(subset):,})")
    print(f"{'='*80}")
    
    # Уникальные значения affinity
    print(f"\nУникальные значения affinity ({subset['affinity'].nunique()}):")
    aff_counts = subset['affinity'].value_counts().head(20)
    for val, count in aff_counts.items():
        print(f"  {str(val):30s}: {count:8,} ({count/len(subset)*100:.2f}%)")
    
    # Уникальные значения processed_measurement
    print(f"\nУникальные значения processed_measurement ({subset['processed_measurement'].nunique()}):")
    pm_counts = subset['processed_measurement'].value_counts().head(20)
    for val, count in pm_counts.items():
        print(f"  {str(val):30s}: {count:8,} ({count/len(subset)*100:.2f}%)")
    
    # Матрица соответствий
    print(f"\nМатрица соответствий (affinity → processed_measurement):")
    crosstab = pd.crosstab(subset['affinity'], subset['processed_measurement'])
    print(crosstab)
    
    # Процент точного соответствия
    exact_match = (subset['affinity'] == subset['processed_measurement']).sum()
    print(f"\nТочное соответствие (affinity == processed_measurement): "
          f"{exact_match:,} ({exact_match/len(subset)*100:.2f}%)")


АНАЛИЗ СПЕЦИАЛЬНЫХ ТИПОВ: fuzzy и bool

Тип: fuzzy (всего: 524,346)

Уникальные значения affinity (3):
  m                             :  190,011 (36.24%)
  h                             :  172,149 (32.83%)
  l                             :  162,186 (30.93%)

Уникальные значения processed_measurement (3):
  m                             :  190,011 (36.24%)
  h                             :  172,149 (32.83%)
  l                             :  162,186 (30.93%)

Матрица соответствий (affinity → processed_measurement):
processed_measurement       h       l       m
affinity                                     
h                      172149       0       0
l                           0  162186       0
m                           0       0  190011

Точное соответствие (affinity == processed_measurement): 524,346 (100.00%)

Тип: bool (всего: 289,382)

Уникальные значения affinity (2):
  1.0                           :  236,494 (81.72%)
  0.0                           :   52,888 (18.28%)

Уника

In [14]:
# 5. Анализ типов с преобразованиями (например, -log KD, kd)
print("=" * 80)
print("АНАЛИЗ ТИПОВ С МАТЕМАТИЧЕСКИМИ ПРЕОБРАЗОВАНИЯМИ")
print("=" * 80)

transform_types = ['-log KD', 'kd', 'log_enrichment', 'ic_50', 'delta_g', 'ddg', 'elisa_mut_to_wt_ratio', 'alphaseq']

for trans_type in transform_types:
    if trans_type not in unique_types:
        continue
        
    subset = agab_df[agab_df['affinity_type'] == trans_type].copy()
    print(f"\n{'='*80}")
    print(f"Тип: {trans_type} (всего: {len(subset):,})")
    print(f"{'='*80}")
    
    subset['affinity_num'] = pd.to_numeric(subset['affinity'], errors='coerce')
    subset['pm_num'] = pd.to_numeric(subset['processed_measurement'], errors='coerce')
    
    numeric_mask = subset['affinity_num'].notna() & subset['pm_num'].notna()
    numeric_subset = subset[numeric_mask].copy()
    
    if len(numeric_subset) == 0:
        print("Нет числовых пар для анализа")
        continue
    
    aff = numeric_subset['affinity_num'].values
    pm = numeric_subset['pm_num'].values
    
    # Тестируем разные преобразования
    print(f"\nЧисловых пар: {len(numeric_subset):,}")
    
    # 1. Прямое равенство
    equal = np.isclose(aff, pm, rtol=1e-10)
    equal_pct = equal.sum() / len(numeric_subset) * 100
    print(f"  1. Прямое равенство: {equal.sum():,} ({equal_pct:.2f}%)")
    
    # 2. -log10 (для kd)
    if trans_type == 'kd' and (aff > 0).all():
        neg_log_pm = -np.log10(aff)
        neg_log_match = np.isclose(neg_log_pm, pm, rtol=1e-5)
        neg_log_pct = neg_log_match.sum() / len(numeric_subset) * 100
        print(f"  2. -log10(affinity): {neg_log_match.sum():,} ({neg_log_pct:.2f}%)")
        
        if neg_log_pct > 90:
            print(f"     ✓ Похоже, что processed_measurement = -log10(affinity) для типа '{trans_type}'")
            # Показываем примеры
            print(f"\n     Примеры:")
            for i in range(min(5, len(numeric_subset))):
                idx = numeric_subset.index[i]
                print(f"       {subset.loc[idx, 'affinity']:15s} → -log10 = {neg_log_pm[i]:.4f} → {subset.loc[idx, 'processed_measurement']:15s}")
    
    # 3. Для "-log KD" - проверяем обратное преобразование (10^(-affinity))
    if trans_type == '-log KD':
        # Если affinity уже является -log KD, то processed_measurement может быть просто affinity
        # Но также может быть какое-то другое преобразование
        # Проверяем, может ли быть, что processed_measurement = affinity - константа
        diff = pm - aff
        if np.std(diff) < 0.1:  # Если разница почти постоянная
            const = np.mean(diff)
            print(f"  2. processed_measurement = affinity + {const:.4f}")
        # Проверяем, может быть processed_measurement = affinity / константа
        ratio = pm / aff
        if (aff != 0).all() and np.std(ratio) < 0.01:
            const_ratio = np.mean(ratio)
            print(f"  3. processed_measurement = affinity * {const_ratio:.4f}")
    
    # 4. log10
    if (aff > 0).all():
        log_pm = np.log10(aff)
        log_match = np.isclose(log_pm, pm, rtol=1e-5)
        log_pct = log_match.sum() / len(numeric_subset) * 100
        if log_pct > 10:  # Показываем только если есть значимое совпадение
            print(f"  4. log10(affinity): {log_match.sum():,} ({log_pct:.2f}%)")
    
    # Показываем примеры для понимания паттерна
    if equal_pct < 100:
        print(f"\n  Примеры преобразований (первые 10):")
        examples = numeric_subset[['affinity', 'processed_measurement', 'affinity_num', 'pm_num']].head(10)
        for idx, row in examples.iterrows():
            ratio = row['pm_num'] / row['affinity_num'] if row['affinity_num'] != 0 else np.nan
            diff = row['pm_num'] - row['affinity_num']
            print(f"    {row['affinity']:20s} → {row['processed_measurement']:20s} "
                  f"(diff: {diff:+.4f}, ratio: {ratio:.4f})")


АНАЛИЗ ТИПОВ С МАТЕМАТИЧЕСКИМИ ПРЕОБРАЗОВАНИЯМИ

Тип: -log KD (всего: 152,401)

Числовых пар: 152,401
  1. Прямое равенство: 18,851 (12.37%)

  Примеры преобразований (первые 10):
    7.759201228882668    → 7.759201228882668    (diff: +0.0000, ratio: 1.0000)
    8.52432881167557     → 8.52432881167557     (diff: +0.0000, ratio: 1.0000)
    8.835354938418874    → 8.835354938418874    (diff: +0.0000, ratio: 1.0000)
    9.32787640751027     → 9.32787640751027     (diff: +0.0000, ratio: 1.0000)
    9.3432385286934      → 9.3432385286934      (diff: +0.0000, ratio: 1.0000)
    9.909124927974114    → 9.909124927974114    (diff: +0.0000, ratio: 1.0000)
    10.387445332552955   → 10.387445332552955   (diff: +0.0000, ratio: 1.0000)
    8.759973792416854    → 8.759973792416854    (diff: +0.0000, ratio: 1.0000)
    9.851688302090514    → 9.851688302090514    (diff: +0.0000, ratio: 1.0000)
    8.92046472668451     → 8.92046472668451     (diff: +0.0000, ratio: 1.0000)

Тип: kd (всего: 6,884)

Число

In [15]:
# 6. Финальная сводка правил преобразования
print("=" * 80)
print("СВОДКА ПРАВИЛ ПРЕОБРАЗОВАНИЯ")
print("=" * 80)

summary_rules = {}

for aff_type in sorted(unique_types):
    subset = agab_df[agab_df['affinity_type'] == aff_type].copy()
    subset['affinity_num'] = pd.to_numeric(subset['affinity'], errors='coerce')
    subset['pm_num'] = pd.to_numeric(subset['processed_measurement'], errors='coerce')
    
    numeric_mask = subset['affinity_num'].notna() & subset['pm_num'].notna()
    numeric_subset = subset[numeric_mask]
    
    rule = {
        'total': len(subset),
        'numeric_pairs': len(numeric_subset),
        'rule': 'не определено'
    }
    
    # Для fuzzy и bool - специальная обработка
    if aff_type == 'fuzzy' or aff_type == 'bool':
        exact_match = (subset['affinity'] == subset['processed_measurement']).sum()
        match_pct = exact_match / len(subset) * 100
        if match_pct > 95:
            rule['rule'] = 'processed_measurement = affinity (прямое копирование)'
        else:
            rule['rule'] = 'категориальное преобразование (требует анализа)'
    elif len(numeric_subset) > 0:
        aff = numeric_subset['affinity_num'].values
        pm = numeric_subset['pm_num'].values
        
        # Проверяем прямое равенство
        if np.isclose(aff, pm, rtol=1e-10).sum() / len(numeric_subset) > 0.95:
            rule['rule'] = 'processed_measurement = affinity (прямое копирование)'
        # Проверяем -log10 для kd
        elif aff_type == 'kd' and (aff > 0).all():
            neg_log_pm = -np.log10(aff)
            if np.isclose(neg_log_pm, pm, rtol=1e-5).sum() / len(numeric_subset) > 0.95:
                rule['rule'] = 'processed_measurement = -log10(affinity)'
        # Для "-log KD" - проверяем специальные случаи
        elif aff_type == '-log KD':
            # Проверяем, может ли быть какое-то смещение или масштабирование
            diff = pm - aff
            if np.std(diff) < 0.1 and np.abs(np.mean(diff)) > 0.01:
                const = np.mean(diff)
                rule['rule'] = f'processed_measurement = affinity + {const:.4f} (приблизительно)'
            else:
                rule['rule'] = 'требует дополнительного анализа (возможно сложное преобразование)'
        # Проверяем другие паттерны
        else:
            rule['rule'] = 'требует дополнительного анализа'
    
    summary_rules[aff_type] = rule
    
    print(f"\n{aff_type:25s}: {rule['rule']}")
    print(f"{' ' * 27}Всего: {rule['total']:,}, числовых пар: {rule['numeric_pairs']:,}")

print("\n" + "=" * 80)


СВОДКА ПРАВИЛ ПРЕОБРАЗОВАНИЯ

-log KD                  : требует дополнительного анализа (возможно сложное преобразование)
                           Всего: 152,401, числовых пар: 152,401

alphaseq                 : требует дополнительного анализа
                           Всего: 131,645, числовых пар: 129,079

bool                     : processed_measurement = affinity (прямое копирование)
                           Всего: 289,382, числовых пар: 289,382

ddg                      : требует дополнительного анализа
                           Всего: 670, числовых пар: 670

delta_g                  : требует дополнительного анализа
                           Всего: 2,725, числовых пар: 2,725

elisa_mut_to_wt_ratio    : processed_measurement = affinity (прямое копирование)
                           Всего: 658, числовых пар: 658

fuzzy                    : processed_measurement = affinity (прямое копирование)
                           Всего: 524,346, числовых пар: 0

ic_50                